What I need to do here to make sure that there is no confusion about the positions of opposite homozygotes is to call SNPs using the joint calling with the 2K matrix.

In [ ]:
source("../../BrusselSprouts/scripts/functions.R")


scripts_dir="/home/jnrunge/data/trd/mapped_reads/scripts/"

initial_timedate=Sys.time()

# what needs to be done is individually, and parallely genotype all crosses, then merge them into a DB, and once call the joint genotyping (once per batch of sequencing, so twice in total)

bam_files=list.files(path="~/data/trd/mapped_reads",pattern="JNRC[0-9]*.bam$",full.names = TRUE)
bam_files=c(bam_files, list.files(path="~/data/trd/mapped_reads",pattern="ChrisC[0-9].bam$",full.names = TRUE))
bam_files=naturalsort(bam_files)
crosses=unlist(lapply(basename(bam_files), getFirst_v2, split="."))

for(cross in crosses){
    print(cross)

    if(file.exists(paste0("/home/jnrunge/data/trd/mapped_reads/",cross,".g.vcf.gz"))){
        print("GVCF exists")
        next
    }
    file.create(running_file<-paste0("~/data/TRD/",cross,".runningGATK"))
    cmds=paste0("cd /home/jnrunge/data/trd/mapped_reads/ && gatk AddOrReplaceReadGroups -I ",cross,".bam -O ",cross,".ReadGroups.bam --RGID ",cross," --RGLB ",cross," --RGPL ILLUMINA --RGPU ",cross," --RGSM ",cross,"")
    cmds=c(cmds,paste0("mv -f ",cross,".ReadGroups.bam ",cross,".bam"))
    cmds=c(cmds,paste0("samtools index ",cross,".bam"))
    cmds=c(cmds,paste0("gatk HaplotypeCaller -R ../../TRD/R64_nucl.fasta -I ",cross,".bam -O ",cross,".g.vcf.gz --emit-ref-confidence GVCF && rm -f ","~/data/TRD/",cross,".runningGATK"))

    sbatch_list=execute_complex_sbatch(cmds, jobname = jobname<-"GATK", scripts_dir = scripts_dir, uniqueRunID = cross, cores="1", mem="32gb", time="long", env="bwaetc", initial_timedate = initial_timedate, jobs_simul = 10, jobs_total = 10)


}

if(exists("sbatch_list")){
    print("Batching up...")
    start_sbatch_list(sbatch_list, 10, jobname, initial_timedate)
    }

while(length(list.files(path = "~/data/TRD", pattern = ".runningGATK$"))>0){
    Sys.sleep(60)
}


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




[1] "ChrisC1"
[1] "GVCF exists"
[1] "ChrisC2"
[1] "GVCF exists"
[1] "ChrisC3"
[1] "GVCF exists"
[1] "ChrisC4"
[1] "GVCF exists"
[1] "ChrisC5"
[1] "GVCF exists"
[1] "ChrisC6"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/GATK-ChrisC6.sbatch"
[1] "ChrisC7"
[1] "GVCF exists"
[1] "ChrisC8"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/GATK-ChrisC8.sbatch"
[1] "YJNRC2"
[1] "GVCF exists"
[1] "YJNRC17"
[1] "GVCF exists"
[1] "YJNRC18"
[1] "GVCF exists"
[1] "Batching up..."
[1] "2 /home/jnrunge/data/trd/mapped_reads/scripts/GATK.list"
[1] "[1] \"Currently running 0 jobs.\""               
[2] "[1] \"2 sbatches remaining! Running 11 more...\""
[3] "[1] \"Submitted batch job 483880\""              
[4] "[1] \"Submitted batch job 483881\""              


In [ ]:
library(readxl)

crosses=list.files("~/data/trd/mapped_reads",pattern = "g.vcf.gz$",full.names = TRUE)

my_crosses=read_xlsx("~/data/trd/Crosses.xlsx", sheet=2)[,c("Short name 1", "Short name 2", "Status", "Colonies in pool")]
chris_crosses=data.frame(sample1=c("ACP","BAP","CCD","ATE","ACK","AKE","BAH","ANG"),
                        sample2=c("BFP","CMP","CPG","YCR","ACV","BAH","CGD","CEI"))
all_used_samples=unique(c(my_crosses$`Short name 1`, my_crosses$`Short name 2`, chris_crosses$sample1, chris_crosses$sample2))
all_used_samples[all_used_samples=="YCR"]=paste0("SACE_",all_used_samples[all_used_samples=="YCR"])
summary(file.exists(paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz")))

all_used_samples[!file.exists(paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz"))]


samples=c(crosses,paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz"))
samples=data.frame(id=unlist(lapply(basename(samples), getFirst_v2, split=".")),file=samples)
fwrite(samples, "~/data/trd/mapped_reads/all_samples.tsv", col.names = FALSE, sep="\t")
head(samples)

In [ ]:
# make BED file (all positions because GATK is retarded)
fai=fread("~/data/TRD/R64_nucl.fasta.fai")
bed=data.frame(chr=fai$V1, from=1, to=fai$V2)
head(bed)
fwrite(bed, "~/data/TRD/R64_nucl.fasta.fai.bed", col.names = FALSE, sep="\t")

In [ ]:
if(file.mtime("~/data/trd/mapped_reads/TRD.vcf.gz")<max(file.mtime(samples$file))){
    # if new samples can be added to the big vcf, then merge again
    
    file.create(running_file<-paste0("~/data/TRD/runningGATKmerge"))
    cmd="cd ~/data/trd/mapped_reads && rm -rf TRD_DB"
    cmd=paste0(cmd, " && gatk GenomicsDBImport --batch-size 200 --genomicsdb-workspace-path TRD_DB --sample-name-map ~/data/trd/mapped_reads/all_samples.tsv -L ~/data/TRD/R64_nucl.fasta.fai.bed")
    cmd=paste0(cmd, " && gatk GenotypeGVCFs -R ../../TRD/R64_nucl.fasta -V gendb://TRD_DB -O TRD.vcf.gz")
    cmd=paste0(cmd, " && bcftools query -l TRD.vcf.gz > TRD.vcf.gz.samples && rm -f ~/data/TRD/runningGATKmerge")
    execute_cmd_sbatch(cmd, mem="32G", cpu="1", time="long", env="bwaetc", jobname="GATK_merge")
    
    while(file.exists(running_file)){
    Sys.sleep(60)
}
    
}


In [ ]:
Crosses=readxl::read_xlsx("~/data/trd/Crosses.xlsx", sheet=2)
head(Crosses<-Crosses[,c("Cross ID","Short name 1","Short name 2")])
tail(Crosses)

In [12]:
gvcf="/home/jnrunge/data/trd/mapped_reads/TRD.vcf.gz"

In [13]:
crosses=unlist(lapply(basename(bam_files), getFirst_v2, split="."))

for(cross in crosses){
    run=FALSE
    print(cross)
    gvcf<-paste0("/home/jnrunge/data/trd/mapped_reads/",cross,".g.vcf.gz")
    if(file.exists(ohfile<-paste0("~/data/trd/mapped_reads/",cross,".hetLoci.gz"))){
        if(file.mtime(gvcf)>file.mtime(ohfile)){
            run=TRUE
            
        }
    }else{
        run=TRUE
    }
    if(run == TRUE){
            file.create(running_file<-paste0("~/data/TRD/Oppo-Homo-Pos/",cross,".runningOHLOCI"))
            cmd=paste0("cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci2.sh ",cross)
            cmd=paste0(cmd," && rm -f ",running_file)
            execute_cmd_sbatch(cmd, mem="8G", cpu="1", time="short", env="bwaetc", jobname = "OHLoci")
        Sys.sleep(1)
            print(cmd)
    }
    }
while(length(list.files(path = "~/data/TRD/Oppo-Homo-Pos", pattern = ".runningOHLOCI$"))>0){
    Sys.sleep(60)
}

[1] "ChrisC1"
[1] "ChrisC2"
[1] "ChrisC3"
[1] "ChrisC4"
[1] "ChrisC5"
[1] "ChrisC6"
[1] "ChrisC7"
[1] "ChrisC8"
[1] "YJNRC2"
[1] "YJNRC17"
[1] "YJNRC18"


In [ ]:
for(s in samples$id){
    if(grepl("YJNRC",s,fixed = TRUE)|grepl("Chris",s,fixed = TRUE)){
        next
    }
    run=FALSE
    print(s)
    if(file.exists(ohfile<-paste0("~/data/trd/mapped_reads/",s,".homLoci.gz"))){
        if(file.mtime(gvcf)>file.mtime(ohfile)){
            run=TRUE
            
        }
    }else{
        run=TRUE
    }
    if(run == TRUE){
            file.create(running_file<-paste0("~/data/TRD/Oppo-Homo-Pos/",s,".runningOHLOCI"))
            cmd=paste0("cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh ",s)
            cmd=paste0(cmd," && rm -f ",running_file)
            execute_cmd_sbatch(cmd, mem="8G", cpu="1", time="short", env="bwaetc", jobname = "OHLoci")
            print(cmd)
        Sys.sleep(1)
        }
    }
while(length(list.files(path = "~/data/TRD/Oppo-Homo-Pos", pattern = ".runningOHLOCI$"))>0){
    Sys.sleep(60)
}

[1] "BAK"
[1] "BAN"
[1] "AKH"
[1] "ABS"
[1] "AMM"
[1] "ABE"
[1] "BFC"
[1] "BID"
[1] "ACT"
[1] "AKV"
[1] "AMD"
[1] "ANI"
[1] "AND"
[1] "ANH"
[1] "AVI"
[1] "ACI"
[1] "ABC"
[1] "ACN"
[1] "AAM"
[1] "ABG"
[1] "AAR"
[1] "ABA"
[1] "BEF"
[1] "AKI"
[1] "ATE"
[1] "BET"
[1] "BTI"
[1] "BQC"
[1] "BMK"
[1] "CRB"
[1] "BKL"
[1] "ABL"
[1] "CCF"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh CCF && rm -f ~/data/TRD/Oppo-Homo-Pos/CCF.runningOHLOCI'"
[1] "Submitted batch job 484897"
[1] "cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh CCF && rm -f ~/data/TRD/Oppo-Homo-Pos/CCF.runningOHLOCI"
[1] "CCC"
[1] "sbatch -c 1 --mem=8G --job-name=OHLoci -p fast -t 11:59:00 --wrap '. ~/activate.sh bwaetc; cd /home/jnrunge/data/trd/mapped_reads/ && sh ~/TRD/01_Mapping/02_OHLoci3.sh CCC && rm -f ~/data/TRD/Oppo-Homo-Pos/CCC.runningOHLOCI'"
[1] "Submitted batch job 4